In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
#os.environ["TF_CPP_VMODULE"] = "gpu_process_state=10,gpu_cudamallocasync_allocator=10"
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        # allowing gpu memory growth
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


2021-11-24 11:20:58.530334: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-24 11:20:58.616357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-24 11:20:58.617201: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-24 11:20:58.619972: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from transformers import InputExample, InputFeatures

model= GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=3)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
#model.

In [5]:
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('Twitter_Data(Relabeled).csv')
dataset.columns=['clean_text','category']
dataset['category']=pd.to_numeric(dataset['category'],errors='coerce')
dataset=dataset.dropna(subset=['category'])
dataset['category']=dataset['category'].astype(int)
train , test =train_test_split(dataset,train_size=0.7)
train.head(10)

,clean_text,category
28893,modi agenda new indiawhat was his agenda 2014 ...,2
130984,\nonly work bogibeel bridge could completed lo...,2
71114,rahul bro work hard and defeat terrorist modi,0
36293,ashoka does not value such gifts honours that ...,1
21422,india blundered not declarled hindu nation in1...,0
89939,european parliament members write modi rebuke ...,0
47077,enters space “super league” modi announces ind...,2
91397,naah naah nothing wrong with being anti modi t...,2
135345,writer takers you that too when producer made ...,0
4854,please investigate the bank role this scam mod...,0


In [6]:
test.head()

,clean_text,category
5861,has forgotten that his party only got abt vote...,2
23359,every modi bhakt like source reddit,1
34834,this the modi tailor launch please watch jai s...,1
13510,india has achieved many feats today and the pr...,2
51187,very much safe under modi bharat mata joy,2


In [7]:
InputExample(guid=None,
             text_a= "Hello, world",
             text_b=None,
             label=1.0)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1.0)

In [8]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'clean_text', 
                                                                           'category')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        try:
          input_dict = tokenizer.encode_plus(
              e.text_a,
              add_special_tokens=True,
              max_length=max_length, # truncates if len(s) > max_length
              return_token_type_ids=True,
              return_attention_mask=True,
              pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
              truncation=True,
          )

          input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
              input_dict["token_type_ids"], input_dict['attention_mask'])

          features.append(
              InputFeatures(
                  input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
              )
          )
        except:
          print("error produced by:")
          print(e)

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'clean_text'
LABEL_COLUMN = 'category'

In [9]:
len(dataset)

162973

In [10]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(16).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(16)

In [ ]:
checkpoint_path = "training_2/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
try:
    model.load_weights(checkpoint_path)
except:
    print("No checkpoint found")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data,callbacks=[cp_callback])


In [ ]:
pred_tweets = ['president is such a lazy person, does not work at all', 'pinapple pizza may just be the worst dish ever created',
               'last night episode was the most engaging in the last two seasons', 'pasta with no sauce and Fettuccine Alfredo']
tf_batch=tokenizer(pred_tweets, max_length=128, padding=True, truncation=True,return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions=tf.nn.softmax(tf_outputs[0],axis=-1)
labels = ['Negative','Neutral','Positive']
label=tf.argmax(tf_predictions,axis=-1)
label=label.numpy()
for i in range(len(pred_tweets)):
    print(pred_tweets[i], ": \n", labels[label[i]])